In [152]:
%load_ext autoreload

%autoreload 2

import cPickle as pickle
from glob import glob
import os
from os.path import join
import random
import numpy as np

import threebody

import logging
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
logging.debug("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Select a model to evaluate

Each block in this section loads up a particular model; the sections below try evaluating it with various parameter sets.

In [ ]:
# fitter_params

d = pickle.load(open("fitter_params.pickle"))
F = threebody.Fitter(**d)

d

In [ ]:
# emcee_params

d = pickle.load(open("emcee_params.pickle"))
F = threebody.Fitter(**d)

d

In [ ]:
# MCMC run

jobid = "4203"
job_dir = glob(os.path.join("emcee-chains",jobid+".*"))[0]

with open(os.path.join(job_dir,"fitter_params.pickle")) as f:
    fitter_params = pickle.load(f)
#fitter_params = pickle.load(open("fitter_params.pickle"))
F = threebody.Fitter(**fitter_params)

fiter_params

In [182]:
# downhill run

#name = "dmtesting"
#name = "newton"
#name = "heavysimple"
#name = "GR"
name = "GRdaily"
d = pickle.load(open(join("downhill-runs",name,"fitter_params.pickle")))
F = threebody.Fitter(**d)

d

{'dmx_span': 1,
 'efac': 1,
 'fdn_range': (1, 5),
 'files': 'tuned2_aligned_pulses_1400_nosun2',
 'fit_pm': True,
 'fit_pos': True,
 'fit_px': True,
 'kopeikin': False,
 'linear_dm': False,
 'linear_jumps': True,
 'parfile': '0337_de435.par',
 'ppn_mode': 'GR',
 'priors': ('dbeta', 'dgamma'),
 't2_astrometry': True,
 'toa_mode': 'pipeline',
 'tzrmjd_middle': 'auto',
 'variable_dm': True,
 'variable_ipm': False}

### Update the database based on fitter results

In [134]:
bpd = threebody.load_best_parameter_database()

p = pickle.load(open("fitter_best_values.pickle"))
#p = pickle.load(open("derivative-best-parameters.pickle"))
#p = pickle.load(open("downhill_best_values.pickle"))
newp = F.best_parameters.copy()

newp.update(p)
try:
    old = F.goodness_of_fit(F.best_parameters)
except KeyError:
    old = np.inf
new = F.goodness_of_fit(newp)
print "existing best parameters:", old
print "best parameters from fitter:", new

existing best parameters: 36691.1758817
best parameters from fitter: 1.47959168284e+12


In [114]:
if new<old:
    print "updating"
    bpd[F.bpd_k] = newp
threebody.save_best_parameter_database(bpd)

updating


### Update the database by looking at the best results from an MCMC run

See also k-downhill-runner , which allows taking an MCMC best position as the start point for a downhill optimizer run.

In [ ]:
jobid = "4203"
job_dir = glob(os.path.join("emcee-chains",jobid+".*"))[0]

In [130]:
with open(os.path.join(job_dir,"fitter_params.pickle")) as f:
    fitter_params = pickle.load(f)
#fitter_params = pickle.load(open("fitter_params.pickle"))
F = threebody.Fitter(**fitter_params)
fitter_params

{'efac': 1,
 'fdn_range': (1, 5),
 'files': 'tuned2_aligned_pulses_1400_nosun',
 'fit_pm': True,
 'fit_pos': True,
 'fit_px': True,
 'kopeikin': False,
 'linear_dm': True,
 'linear_jumps': True,
 'parfile': '0337_tempo2_px_optical_dmcorr.par',
 'ppn_mode': 'heavysimple',
 'priors': ('dbeta', 'dgamma'),
 't2_astrometry': True,
 'toa_mode': 'pipeline',
 'tzrmjd_middle': 'auto'}

In [131]:
best_so_far = np.load(os.path.join(job_dir,"best_so_far.npy"))
newp = F.best_parameters.copy()
for i,p in enumerate(F.parameters):
    newp[p] = best_so_far[i]

In [132]:
old = F.goodness_of_fit()
new = F.goodness_of_fit(newp)
print "existing best parameters:", old
print "best parameters from MCMC:", new

existing best parameters: 36212.1915594
best parameters from MCMC: 36194.4002061


In [133]:
bpd = threebody.load_best_parameter_database()
if new<old:
    bpd[frozenset(F.args.iteritems())] = newp
threebody.save_best_parameter_database(bpd)

### Update the database for a fitter by trying values for all known configurations

In [183]:
#d = pickle.load(open("fitter_params.pickle"))
#d = pickle.load(open("downhill-runs/WSRT/fitter_params.pickle"))
#d = pickle.load(open("emcee-chains/32534.nimrod/fitter_params.pickle"))
#d = pickle.load(open("fake_toas_fitter.pickle"))
#d = pickle.load(open("forecaster_params.pickle"))
#d['files'] = 'fake-uniform'
#d['kopeikin'] = True
#d['fit_pm'] = True
#d['fit_px'] = True

#d = pickle.load(open("sensitivity_params.pickle"))
#d = pickle.load(open("plot_params.pickle"))
#d = pickle.load(open("scratch_params.pickle"))
#print d
#F = threebody.Fitter(**d)
#print F.args
if "j_NCY1400" not in F.best_parameters:
    F.best_parameters["j_NCY1400"] = 0.25*2.7e-3
if "delta" not in F.best_parameters:
    F.best_parameters['delta'] = 0
try:
    old = F.goodness_of_fit(F.best_parameters)
except KeyError:
    print "Missing best parameter values!"
    old = np.inf
print "existing best parameters:", old
best = old
best_p = F.best_parameters.copy()
itms = bpd.items()
random.shuffle(itms)
def sim(i):
    n = 0
    k,v = i
    dk = dict(k)
    for p, v in dk.items():
        if p not in d or d[p]!=v:
            n += 1
    for p, v in d.items():
        if p not in dk:
            n += 1
    return n
itms.sort(key=sim)
for k, p in itms: 
    newp = F.best_parameters.copy()
    newp.update(p)
    new = F.goodness_of_fit(newp)
    print "best parameters from database:", new
    if new<best:
        best = new
        best_p = newp
        if True:
            bpd = threebody.load_best_parameter_database()
            bpd[frozenset(F.args.iteritems())] = best_p
            threebody.save_best_parameter_database(bpd)


INFO:threebody:Missing key in parameters: asini_i
INFO:threebody:Missing key in parameters: pb_i
INFO:threebody:Missing key in parameters: eps1_i
INFO:threebody:Missing key in parameters: eps2_i
INFO:threebody:Missing key in parameters: tasc_i
INFO:threebody:Missing key in parameters: acosi_i
INFO:threebody:Missing key in parameters: q_i
INFO:threebody:Missing key in parameters: asini_o
INFO:threebody:Missing key in parameters: pb_o
INFO:threebody:Missing key in parameters: eps1_o
INFO:threebody:Missing key in parameters: eps2_o
INFO:threebody:Missing key in parameters: tasc_o
INFO:threebody:Missing key in parameters: acosi_o
INFO:threebody:Missing key in parameters: delta_lan


existing best parameters: 5.56061586526e+30
best parameters from database: 185206441194.0
best parameters from database: 34557.0781732
best parameters from database: 230530988645.0


KeyboardInterrupt: 

In [ ]:
best_p

In [ ]:
bpd[frozenset(F.args.iteritems())] = best_p
threebody.save_best_parameter_database(bpd)

In [ ]:
bpd.items()

In [ ]:
newp

In [ ]:
for k in bpd.keys():
    d = dict(k)
    if False and 'shapiro' not in d:
        d['shapiro'] = True
        bpd[frozenset(d.iteritems())] = bpd[k]
        d['shapiro'] = False
        bpd[frozenset(d.iteritems())] = bpd[k]
    if d['parfile'] == '0337_tempo2_pm.par':
        d['parfile'] = '0337_tempo2_px.par'
        nk = frozenset(d.iteritems())
        if nk not in bpd:
            bpd[nk] = bpd[k]

In [ ]:
for k in F.best_parameters:
    if k not in p:
        print "missing from p:", k, F.best_parameters[k]
    elif F.best_parameters[k]!=p[k]:
        print "different:", k, F.best_parameters[k], p[k]
        

In [ ]:
for k in bpd.keys():
    if k==frozenset(d.iteritems()):
        bpd[k].pop('pm_x',None)
        bpd[k].pop('pm_y',None)
        bpd[k].pop('f1',None)

In [ ]:
F.parameters

In [ ]:
paper1 = dict(files="0337+17-scott-2013-06-06",
                         parfile="0337_tempo2_nobinary.par",
                         tzrmjd_middle='auto',
                         fit_pos=False,
                         fit_pm=False,
                         fit_px=False,
                         t2_astrometry=True,
                         kopeikin=False,
                         priors=(),
                         ppn_mode=None)

In [ ]:
F = threebody.Fitter(**paper1)

In [ ]:
bpd[frozenset(F.args.iteritems())]

In [ ]:
for ot in [['AO1350','AO1440'],['GBT1500'],['WSRT1400']]:
    args = F.args.copy()
    args['only_tels'] = tuple(ot)
    bpd[frozenset(args.iteritems())] = bpd[frozenset(F.args.iteritems())]

## Fill in missing parameter values

In [ ]:
np.c